Deep Learning - Assignment 4
=============

The goal of this assignment is make the <b>neural network convolutional</b>.

_____

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [9]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  x = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(x)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [10]:
num_steps =1001

with tf.Session(graph=graph) as session:
  session.run(tf.global_variables_initializer())
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {x : batch_data, y : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.833553
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 100: 0.998327
Minibatch accuracy: 62.5%
Validation accuracy: 71.6%
Minibatch loss at step 200: 0.928417
Minibatch accuracy: 68.8%
Validation accuracy: 78.0%
Minibatch loss at step 300: 0.409400
Minibatch accuracy: 87.5%
Validation accuracy: 79.0%
Minibatch loss at step 400: 0.234134
Minibatch accuracy: 93.8%
Validation accuracy: 80.1%
Minibatch loss at step 500: 0.608130
Minibatch accuracy: 81.2%
Validation accuracy: 80.8%
Minibatch loss at step 600: 0.378692
Minibatch accuracy: 93.8%
Validation accuracy: 82.0%
Minibatch loss at step 700: 1.018403
Minibatch accuracy: 68.8%
Validation accuracy: 82.3%
Minibatch loss at step 800: 0.693525
Minibatch accuracy: 75.0%
Validation accuracy: 82.7%
Minibatch loss at step 900: 0.850923
Minibatch accuracy: 87.5%
Validation accuracy: 83.1%
Minibatch loss at step 1000: 0.418812
Minibatch accuracy: 87.5%
Validation accuracy: 81.8%

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation of stride 2 and kernel size 2.

---

In [12]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()
with graph.as_default():
  x = tf.placeholder( tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  layer1_weights = tf.Variable(tf.truncated_normal( [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal( [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(  [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(  [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(pool + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(pool + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  logits = model(x)
  loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 1001

with tf.Session(graph=graph) as session:
  session.run(tf.global_variables_initializer())
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {x : batch_data, y : batch_labels}
    _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(   valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.233755
Minibatch accuracy: 6.2%
Validation accuracy: 9.9%
Minibatch loss at step 100: 1.509123
Minibatch accuracy: 43.8%
Validation accuracy: 45.8%
Minibatch loss at step 200: 0.950480
Minibatch accuracy: 81.2%
Validation accuracy: 75.5%
Minibatch loss at step 300: 0.492849
Minibatch accuracy: 75.0%
Validation accuracy: 78.3%
Minibatch loss at step 400: 0.253169
Minibatch accuracy: 93.8%
Validation accuracy: 79.3%
Minibatch loss at step 500: 0.891596
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%
Minibatch loss at step 600: 0.340021
Minibatch accuracy: 87.5%
Validation accuracy: 82.1%
Minibatch loss at step 700: 0.771184
Minibatch accuracy: 81.2%
Validation accuracy: 82.8%
Minibatch loss at step 800: 0.627990
Minibatch accuracy: 75.0%
Validation accuracy: 83.5%
Minibatch loss at step 900: 0.714570
Minibatch accuracy: 81.2%
Validation accuracy: 83.6%
Minibatch loss at step 1000: 0.468226
Minibatch accuracy: 87.5%
Validation accuracy: 84.1%
T

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [25]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
learning_rate= 0.5

graph = tf.Graph()
with graph.as_default():
  x = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  keep_prob = tf.placeholder(tf.float32)
  
  layer1_weights = tf.Variable(tf.random_normal( [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.random_normal( [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.zeros([depth]))
  layer3_weights = tf.Variable(tf.random_normal( [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.zeros([num_hidden])) 
  layer4_weights = tf.Variable(tf.random_normal( [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.zeros([num_labels])) 
  
  def hidden_layer(input,w,b):
        conv = tf.nn.conv2d(input, w, [1, 1, 1, 1], padding='SAME')
        pool = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        return tf.nn.relu(pool + b)
        
  def model(data):
    hl1=hidden_layer(data,layer1_weights,layer1_biases)
    hl2=hidden_layer(hl1,layer2_weights,layer2_biases)
    shape = hl2.get_shape().as_list()
    reshape = tf.reshape(hl2, [shape[0], shape[1] * shape[2] * shape[3]])
    hl3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    drop = tf.nn.dropout(hl3, keep_prob)
    return tf.matmul(drop, layer4_weights) + layer4_biases
  
  logits = model(x)
  loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
 
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    
  
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 3001
with tf.Session(graph=graph) as session:
  session.run(tf.global_variables_initializer())
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {x : batch_data, y : batch_labels, keep_prob: 1.0}
    feed_dict_w_drop = {x : batch_data, y : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict_w_drop)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy( valid_prediction.eval(feed_dict=feed_dict), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(feed_dict=feed_dict), test_labels))

ValueError: Tensor("ExponentialDecay:0", shape=(), dtype=float32) must be from the same graph as Tensor("beta2_power/read:0", shape=(), dtype=float32).